In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pyodbc 
db = pyodbc.connect('Driver={SQL Server};'
                      'Server=PGI-M2M;'
                      'Database=M2MDATA04;'
                      'Trusted_Connection=yes;')

In [2]:
query = 'select fpartno, fpartrev, flot, fonhand  from dbo.inonhd'

In [3]:
v = pd.read_sql_query(query , db)

In [4]:
v['fpartno'] = v['fpartno'].str.replace(' ', '')
v['flot'] = v['flot'].str.replace(' ', '')
v['fonhand'] = v['fonhand'].replace(' ', '')
v['fpartrev'] = v['fpartrev'].replace(' ', '')

v.replace('', np.nan, inplace=True)
v.applymap(lambda x: x.rstrip() if type(x)==str else x)
v = v[pd.notnull(v['flot'])]
v = v.drop_duplicates('flot',keep = 'first')

In [5]:
v

,fpartno,fpartrev,flot,fonhand
10194,100372-LF,,PG1,1.0
10413,240476-LF,,13,1000.0
10414,42023-LF,,1C,390.0
10415,24489-LF,,MW62620181130,28.0
10416,42343-LF,,1G,2.0
10417,42342-LF,,1H,8.0
10418,24702-LF,,RB62620181159,2570.0
10419,24705-LF,,RB62620181207,2140.0
10420,24706-LF,,RB62620181212,27.0
10421,24711-LF,,RB62620181222,85.0


In [6]:
query2 = 'select fpartno, frev, fdescript, FPRODCL, FSOURCE, flocate1 from dbo.inmastx'

In [7]:
c = pd.read_sql_query(query2 , db)

In [8]:
c.applymap(lambda x: x.rstrip() if type(x)==str else x)
c.replace('', np.nan, inplace=True)

c.applymap(lambda x: x.rstrip() if type(x)==str else x)

c['fdescript'] = c['fdescript'].str.replace(' ', '')
c['fpartno'] = c['fpartno'].str.replace(' ', '')
c['FSOURCE'] = c['FSOURCE'].str.replace(' ', '')
c['frev'] = c['frev'].str.replace(' ', '')
c['FPRODCL'] = c['FPRODCL'].str.replace(' ', '')



c = c[pd.notnull(c['fdescript'])]
c=c[c['FSOURCE'].str.contains ("S")]
c[c.fdescript != '**DONOTUSE**']
c=c[c['FPRODCL'].str.contains ("70", "40")]


c.drop(['FPRODCL', 'FSOURCE', 'flocate1'], axis=1, inplace=True)

In [9]:
c = c.drop_duplicates('fpartno',keep = 'first')

In [10]:
v = c.merge(v, on="fpartno", how= 'inner')

In [11]:
v['frev'].fillna(v['fpartno'])
v.drop(['fpartrev'], axis=1, inplace=True)

In [12]:
query3 = 'select fdescript, fvpartno from dbo.poitem'
p= pd.read_sql_query(query3 , db)


p['fdescript'] = p['fdescript'].replace(' ', '')
p['fvpartno'] = p['fvpartno'].replace(' ', '')
p = p[pd.notnull(p['fvpartno'])]
p = p[pd.notnull(p['fdescript'])]

In [13]:
v.to_csv('061019.csv', index = False)

In [14]:
conv = dict(zip(p['fdescript'].values, p['fvpartno'].values))

In [16]:
#v['fvpartno']= .fillna(v['fdescript'].apply(lambda x: conv.get(x)))
#v['fvpartno'] = p['fvpartno'].apply(lambda x: conv.get(x))
#v['fvpartno'] = pd.merge(v['fvpartno'], p['fvpartno'], on="fdescript", how= 'inner')
#new_df = pd.merge(df1,df2, on="my_column")
#v['fvpartno'] = p['fvpartno'].fillna(p['fdescript'])
#v['fvpartno'] = v['fdescript'].where(p['fdescript'].isnull(), p['fdescript'])

new_df = pd.merge(v, p, on = 'fdescript', how= 'left')

In [17]:
new_df

,fpartno,frev,fdescript,flot,fonhand,fvpartno
0,61132-LF,,"SMIC,AD660ARZ(SO-24),PBFREE",350,44.0,NaN
1,60575-LF,,"SMIC,LMC6042IM/NOPB,SO-8,PBFREE",FB31520198321,39.0,NaN
2,60575-LF,,"SMIC,LMC6042IM/NOPB,SO-8,PBFREE",FB31520198322,936.0,NaN
3,330210-LF,,"SMCRYSTAL,40MHZ,CLOCKOSC,PBFREE",FB81620189002,40.0,NaN
4,21160-LF,,"SMRES,16.2K,1%,1/10W,0603",EP713186142,36.0,CR0603-10W-1622FT
5,21160-LF,,"SMRES,16.2K,1%,1/10W,0603",EP713186142,36.0,ERJ-3EKF1622V
6,21160-LF,,"SMRES,16.2K,1%,1/10W,0603",2H1,4120.0,CR0603-10W-1622FT
7,21160-LF,,"SMRES,16.2K,1%,1/10W,0603",2H1,4120.0,ERJ-3EKF1622V
8,21160-LF,,"SMRES,16.2K,1%,1/10W,0603",EP713186141,37.0,CR0603-10W-1622FT
9,21160-LF,,"SMRES,16.2K,1%,1/10W,0603",EP713186141,37.0,ERJ-3EKF1622V


In [ ]:
v

In [18]:
p

,fdescript,fvpartno
0,CHOKE ...,
1,"ENCLOSURE,TOP,GMRC ...",
2,"ENCLOSURE,BOTTOM,GMRC/OSWC/GMPIO ...",
3,"LABEL,GMRC ...",
4,"CLIP,SPEAKER,GMRC ...",
5,"SMIC,PIC16F648A-I/SO,PROGRAMMED ...",
6,"CONN,10 PIN,MOLEX-TYPE,90* ...",
7,"ENCLOSURE,TOP,GMRC ...",
8,"CRYSTAL,4MHZ,RESONATOR",ZTT-4.00MG
9,"SMCAP,3.3,50V,ALUM,ELEC",EEV-HB1H3R3R
